# Author: Trevor Dorn-Wallenstein
# 11/15/17
# Let's design survey fields of $h+\chi$ Persei that cover a maximum number of a given set of stars.

In [1]:
import numpy as np, astropy.io.fits as fits, matplotlib.pyplot as plt
from astropy.table import Table
from scipy.optimize import minimize
%matplotlib inline

In [2]:
member_hdu = fits.open('cluster_members.fits')
member_table = Table(member_hdu[1].data)
OB_table = member_table[member_table['SpT'] <= 20]

/Users/carina/anaconda/lib/python3.5/site-packages/astropy/table/column.py:928: RuntimeWarning: invalid value encountered in less_equal
  return getattr(self.data, oper)(other)


In [3]:
class Star:
    
    def __init__(self, ra, dec):
        """
        Parameters
        ----------
        ra : float
            Right ascension in decimal degrees
        dec : float
            Declination in decimal degrees
        """
        
        self.ra = ra
        self.dec = dec
        
        #Keeps track of how well covered this star is by our survey
        self.score = 0
        self.field_list = []
        
    def count_fields_star_in(self,test_field_list):
        """
        Counts how many fields this star is in.
        
        Parameter
        ---------
        test_field_list : list
            List of Field objects.
        
        Returns
        -------
        N_fields : int
            Number of fields this star is in.
        """
        
        N_fields = 0
        
        for field in test_field_list:
            
            if field.check_star_in_field(self):
                
                N_fields += 1
                self.field_list.append(field)
        
        return N_fields
    
    def score_star(self,overlap_bonus = 0.25):
        """
        If star is in one field, it gets 1 point. If its in more than one field, it gets 
        1 + overlap_bonus points. Must run count_fields_star_in first.
        
        Parameter
        ---------
        overlap_bonus : float
            bonus you want to give to stars that appear in multiple fields
        
        Returns
        -------
        score : float:
            This star's score!
        """
        
        if len(self.field_list) > 1:
            self.score = 1.0 + overlap_bonus
        elif len(self.field_list) == 1:
            self.score = 1
        else:
            self.score = 0
        
        return self.score
        
    def reset(self):
        """
        Resets field_list and score to empty for a new run
        """
        
        self.score = 0
        self.field_list = []

In [4]:
OB_list = [Star(ra,dec) for ra,dec in zip(OB_table['RAJ2000'],OB_table['DEJ2000'])]

In [5]:
class Field:
    
    def __init__(self, ra, dec, size):
        """
        Parameters
        ----------
        ra : float
            Right ascension of center in decimal degrees
        dec : float
            Declination of center in decimal degrees
        size : float
            Size of square region on a side, arcminutes
        """
        
        self.ra = ra
        self.dec = dec
        self.size = size
    
    @classmethod
    def from_star_list(cls, star_list, size, random = True, i = 0):
        """
        Initialize field from list of stars. If random, will choose randomly. Otherwise
        it will just star_list[i]
        
        Parameters
        ----------
        star_list : list
            List of Star objects
        size : float
            Size of square region on a side, arcminutes
        random : bool
            If true, chooses randomly from star_list. Otherwise, chooses ith entry
        i : int
            If random is false, chooses star_list[i]
        
        """
        
        if random:
            star_init = np.random.choice(star_list)
        else:
            star_init = star_list[i]

        field = cls(star_init.ra,star_init.dec,size)

        return field
        
    def check_star_in_field(self,star):
        """
        Checks if a given star is in the field.
        
        Parameter
        ---------
        star : Star
            Star object
        
        Returns
        -------
        in_field : bool
            True if star is in the field, false if not
        """

        delta_dec_arcmin = np.abs(star.dec-self.dec) * 60.0

        delta_ra_arcmin = np.abs(star.ra - self.ra) * np.cos(star.dec * np.pi/180) * 60.0

        if np.all(np.array(delta_ra_arcmin,delta_dec_arcmin) < self.size/2.0):

            return True

        return False
    
    def count_stars_in_field(self,star_list):
        """
        Counts the number of stars in this field.
        
        Parameter
        ---------
        star_list : list
            List of Star objects.
        
        Returns
        -------
        N_in_field : int
            Number of stars in the field from star_list
        """
        
        N_in_field = 0
        
        for star in star_list:
            
            if self.check_star_in_field(star):
                
                N_in_field += 1
        
        return N_in_field
    
    def to_region_string(self, star_list = None):
        """
        Exports this field to a ds9 region
        
        Parameter
        ---------
        star_list : list
            list of Star objects. Counts the number of stars in the field, exports as text
        
        Returns
        -------
        region_string : str
            a ds9 compatible region string.
        """
        
        region_string = "box({0},{1},{2}',{2}',0)".format(self.ra,self.dec,self.size)
        
        if star_list is not None:
            
            region_string += ' # text={{}}'.format(self.count_stars_in_field(star_list))
            
        return region_string

In [6]:
def score_survey(star_list,field_list,overlap_bonus):
    """
    Given the stars you want and a set of fields, scores the stars, sums it up.
    
    Parameters
    ----------
    star_list : list
        list of Star objects
    field_list : list
        list of Field objects
    overlap_bonus
        
    Returns
    -------
    score : float
        Sum of the scores of all of the stars
    """
    
    score = 0.0
    
    for star in star_list:
        
        star.reset()
        
        star.count_fields_star_in(field_list)
        
        score += star.score_star(overlap_bonus = overlap_bonus)
        
    return score

In [7]:
def make_survey_and_score(theta,args):
    """
    Wrapper for score_survey. Makes a new field_list, then scores the stars in star_list.
    
    Parameter
    ---------
    theta : list
        Should be a list of length N_fieldx2, with first N_field entries being RAs, next 
        N_field entries being Decs.
    args : tuple
        First entry should be a list of star objects in your survey. Second entry
        should be the size of the field of view of the camera in arcminutes. Third entry 
        should be the overlap_bonus
        
    Returns
    -------
    survey_score : float
        Score of this particular survey
    """
    
    try:
        assert len(theta) % 2 == 0
    except AssertionError as e:
        raise AssertionError("Length of theta should be N_fields x 2!")
    
    n_fields = len(theta) // 2
    ras = theta[:n_fields]
    decs = theta[n_fields:]
    
    star_list = args[0]
    field_size = args[1]
    overlap_bonus = args[2]
    
    field_list = [Field(ra,dec,field_size) for ra,dec in zip(ras,decs)]
    
    return score_survey(star_list,field_list,overlap_bonus)

In [8]:
def f_min(theta,args):
    
    return -1*make_survey_and_score(theta,args)

In [9]:
def design_survey(star_list, N_fields, N_start = None, overlap_bonus = 0.1):
    """
    Designs a survey by progressively eliminating the field that contributes to the survey
    value the least
    
    Parameters
    ----------
    star_list : list
        list of Star objects
    N_fields : int
        The target number of fields in the survey
    N_start : int
        If given, starts the survey with N_start random fields chosen from star_list
    overlap_bonus : float
        bonus you want to give to stars that appear in multiple fields
        
    Returns
    -------
    field_list : list
        list of Field objects that remain.
    """
    
    #Initialize fields from star list
    if N_start is not None:
        field_list = np.array([Field.from_star_list(star_list=star_list,size=3.0) for j in range(N_start)])
    else:
        field_list = np.array([Field.from_star_list(star_list=star_list,size=3.0,random=False,i=j) for j in range(len(star_list))])
    
    while len(field_list) != N_fields:
        
        scores = []
        
        for i,field in enumerate(field_list):
            
            #Get rid of one field
            trial_list = field_list[field_list != field]
            #Score the survey
            this_score = score_survey(star_list,trial_list,overlap_bonus)
            scores.append(this_score)
            
        field_list = np.delete(field_list,np.argmax(scores))
        print(len(field_list))
        
    return field_list

In [10]:
def optimize_survey(star_list,field_list,overlap_bonus):
    """
    Designs a survey that maximizes the survey score
    
    Parameter
    ---------
    star_list : list
        List of Star objects that you want to measure
    field_list : list
        List of initial fields
    overlap_bonus : float
            bonus you want to give to stars that appear in multiple fields
    Returns
    -------
    field_list : list
        Final list of Field objects
    score : float
        Score of the survey
    """
    
    N_fields = len(field_list)
    
    #Give it a list of ras, and then decs
    theta_0 = []
    
    for field in field_list:
        theta_0.append(field.ra)
        
    for field in field_list:
        theta_0.append(field.dec)
    
    field_size = field_list[0].size
        
    res = minimize(f_min,theta_0,args=([star_list,field_size,overlap_bonus]))
    
    theta = res['x']
    
    score = make_survey_and_score(theta,(star_list,field_size,overlap_bonus))
    
    ras = theta[:N_fields]
    decs = theta[N_fields:]
    
    field_list = [Field(ra,dec,field_size) for ra,dec in zip(ras,decs)]
    
    region_string_list = [f.to_region_string(star_list = star_list) for f in field_list]
    
    return field_list,region_string_list,score

In [11]:
scores = []
surveys = []
overlap = 0.1
for i in range(50):
    if i % 5 == 0:
        print(i+1)
    test_field_list = design_survey(OB_list,30,N_start=50)
    test_survey = optimize_survey(OB_list,test_field_list,overlap)
    test_score = test_survey[2]
    scores.append(test_score)
    surveys.append(test_survey)

1
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
6
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
11
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
16
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
49
48
47
46
45
44
43
42
41
40


In [12]:
the_survey = surveys[np.argmax(scores)]

In [15]:
start_string = '# Region file format: DS9 version 4.1 \nglobal color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \nicrs'

for field in the_survey[0]:
    
    reg_str = field.to_region_string()
    
    start_string += ' \n'
    
    start_string += reg_str

with open('field_reg.reg','w') as f:
    f.write(start_string)

start_string

'# Region file format: DS9 version 4.1 \nglobal color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \nicrs \nbox(35.1575,56.97829999999999,3.0\',3.0\',0) \nbox(34.442299999999996,57.08899999999999,3.0\',3.0\',0) \nbox(35.818099999999994,56.96619999999999,3.0\',3.0\',0) \nbox(34.304199999999994,57.40939999999999,3.0\',3.0\',0) \nbox(34.742,57.17099999999999,3.0\',3.0\',0) \nbox(35.326699999999995,57.157399999999996,3.0\',3.0\',0) \nbox(35.6804,57.05259999999999,3.0\',3.0\',0) \nbox(34.799299999999995,57.11139999999999,3.0\',3.0\',0) \nbox(35.5821,57.04149999999999,3.0\',3.0\',0) \nbox(35.238299999999995,56.889599999999994,3.0\',3.0\',0) \nbox(34.83879999999999,57.076899999999995,3.0\',3.0\',0) \nbox(34.92059999999999,57.16479999999999,3.0\',3.0\',0) \nbox(34.7422,57.07869999999999,3.0\',3.0\',0) \nbox(35.04599999999999,56.90699999999999,3.0\',3.0\',0) \nbox(34.87219999999999,57.07359999999999,3.0\

In [14]:
the_survey

([<__main__.Field at 0x10e183f60>,
 ["box(35.1575,56.97829999999999,3.0',3.0',0) # text={}",
  "box(34.442299999999996,57.08899999999999,3.0',3.0',0) # text={}",
  "box(35.818099999999994,56.96619999999999,3.0',3.0',0) # text={}",
  "box(34.304199999999994,57.40939999999999,3.0',3.0',0) # text={}",
  "box(34.742,57.17099999999999,3.0',3.0',0) # text={}",
  "box(35.326699999999995,57.157399999999996,3.0',3.0',0) # text={}",
  "box(35.6804,57.05259999999999,3.0',3.0',0) # text={}",
  "box(34.799299999999995,57.11139999999999,3.0',3.0',0) # text={}",
  "box(35.5821,57.04149999999999,3.0',3.0',0) # text={}",
  "box(35.238299999999995,56.889599999999994,3.0',3.0',0) # text={}",
  "box(34.83879999999999,57.076899999999995,3.0',3.0',0) # text={}",
  "box(34.92059999999999,57.16479999999999,3.0',3.0',0) # text={}",
  "box(34.7422,57.07869999999999,3.0',3.0',0) # text={}",
  "box(35.04599999999999,56.90699999999999,3.0',3.0',0) # text={}",
  "box(34.87219999999999,57.07359999999999,3.0',3.0',0)